In [ ]:
from binance_script import get_balance as binance_balance, client
from gate_script import balance as gate_balance
from wallet_script import balance_usdt as wallet_balance
import os
import pandas as pd
from datetime import datetime

filename = 'balance_vs_btc.csv'
total_binance, usdt_idr_rate = binance_balance()
total_gate = gate_balance

os.system('clear')
print("=== BINANCE ===")
print(f"Total Asset in USDT: {format(total_binance, ',.0f')}")
print(f"Total Asset in IDR: {format(total_binance * usdt_idr_rate, ',.0f')}")

print("\n=== GATE.IO ===")
print(f"Total Asset in USDT: {total_gate}")
print(f"Total Asset in IDR: {format(total_gate * usdt_idr_rate, ',.0f')}")

print("\n=== WALLET ===")
print(f"Total Asset in USDT: {format(wallet_balance, ',.0f')}")
print(f"Total Asset in IDR: {format(wallet_balance * usdt_idr_rate, ',.0f')}")

print("\n=== Bitget ===")
manta_bitget = 325
manta_price = client.ticker_price("MANTAUSDT")
total_bitget = manta_bitget * float(manta_price['price'])
print(f"Total Asset in USDT: {format(total_bitget, ',.0f')}")
print(f"Total Asset in IDR: {format(total_bitget * usdt_idr_rate, ',.0f')}")


print("\n=== TOTAL ===")
total_usdt = total_binance + total_gate + wallet_balance + total_bitget
total_idr = total_usdt * usdt_idr_rate

btc_price = client.ticker_price("BTCUSDT")['price']
total_btc = total_usdt / float(btc_price)
print("BTC Price: ", btc_price)
print(f"Total Asset in USDT: {format(total_usdt, ',.0f')}")
print(f"Total Asset in IDR: {format(total_idr, ',.0f')}")
print(f"Total Asset in BTC: {format(total_btc, ',.8f')}")


# Check if the file exists
file_exists = os.path.isfile(filename)

df = pd.DataFrame({
    'Date': [datetime.now()],
    'BTC_Price': [btc_price],
    'Binance_USDT': [total_binance],
    'Gate_USDT': [total_gate],
    'Other_USDT': [total_bitget + wallet_balance],
    'Total_BTC': [total_btc],
    'Total_USDT': [total_usdt],
    'Total_IDR': [total_idr]
})

# Append the DataFrame to the CSV file
df.to_csv(filename, mode='a', header=not file_exists, index=False)

In [ ]:
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt

def millions(x, pos):
    'The two args are the value and tick position'
    return '%1.1fM' % (x * 1e-6)

df = pd.read_csv(filename)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
formatter = FuncFormatter(millions)

# Create a figure and a set of subplots
fig, axs = plt.subplots(3, 1, figsize=(10, 10))

# Plot the BTC price data on the first subplot
axs[0].plot(df.index, df['BTC_Price'], color='g')
axs[0].set_ylabel('BTC Price')
axs[0].set_title('BTC Price Over Time')

# Plot the Binance USDT data on the second subplot
axs[1].plot(df.index, df['Binance_USDT'], color='r', label='Binance USDT')
axs[1].set_ylabel('Binance USDT')
axs[1].set_title('Binance and Gate io USDT Value Over Time')
axs[1].legend(loc='upper left')


# Create a second y-axis for the second subplot that shares the same x-axis
ax2 = axs[1].twinx()


# Plot the Gate io USDT data on the second y-axis of the second subplot
ax2.plot(df.index, df['Gate_USDT'], color='b', label='Gate USDT')
ax2.set_ylabel('Gate USDT')
ax2.legend(loc='upper right')

# Plot the USDT data on the second subplot
axs[2].plot(df.index, df['Total_USDT'], color='r', label='USDT')
axs[2].set_ylabel('Value in USDT')
axs[2].set_title('USDT and IDR Value Over Time')
axs[2].legend(loc='upper left')

# Create a second y-axis for the second subplot that shares the same x-axis
ax3 = axs[2].twinx()

# Plot the IDR data on the second y-axis of the second subplot
ax3.plot(df.index, df['Total_IDR'], color='b', label='IDR')
ax3.set_ylabel('Value in IDR')
ax3.legend(loc='upper right')

# Apply the formatter to the y-axis of the second plot
ax3.yaxis.set_major_formatter(formatter)


# Adjust the layout
plt.tight_layout()

# Show the plot
plt.show()